In [15]:
import faiss
from extract_features import mySigLipModel
from display_image import ImageDisplay
import time
import numpy as np
import requests
from PIL import Image
import json

In [21]:
image_path = './indexed_100k-200k/siglip_image_urls-100k-200k.json'
index_path = './indexed_100k-200k/siglip-image-index-100k-200k.bin'
dataset_caption_path = './dataset/SBU_captioned_photo_dataset_captions.txt'
dataset_url_path = './dataset/SBU_captioned_photo_dataset_urls.txt'

In [6]:
#load in the embedding extractor
extractor = mySigLipModel()
#load in image displayer
display = ImageDisplay()
#load in indexer
index = faiss.read_index(index_path)

In [24]:
testing_size = 100
k_list = [3, 5, 10, 20, 50]

In [22]:
with open(dataset_caption_path, 'r') as f:
    captions = f.readlines()[:testing_size]
with open(dataset_url_path, 'r') as f:
    urls = f.readlines()[:testing_size]
with open(image_path, 'r') as f:
    train_image_urls = json.load(f)

In [9]:
# select (testing_size * 1.2) images from the dataset in random
selecting_size = int(testing_size * 1.2)  # 20% more for the case that some images are not valid
np.random.seed(0)
selected_indices = np.random.choice(len(captions), selecting_size, replace=False)
selected_captions = [captions[i] for i in selected_indices]
selected_urls = [urls[i] for i in selected_indices]

In [25]:
# compute the recall@k
for k in k_list:
    recall = 0
    count = 0
    for i in range(selecting_size):
        url = selected_urls[i]
        caption = selected_captions[i]
        # check if the image is valid
        try:
            frame = Image.open(requests.get(url, stream=True).raw)
            count += 1
        except:
            continue
        query_embedding = extractor.get_text_embedding(caption)
        # search the k nearest neighbors
        D, I = index.search(query_embedding, k)
        # check if the caption of the image is in the k nearest neighbors
        result_urls = [train_image_urls[j].strip() for j in I[0]]
        # print('Query: {}'.format(caption))
        # print('Query url: {}'.format(url))
        # print('Result urls:')
        # for result_url in result_urls:
        #     print(result_url)
        if url in result_urls:
            recall += 1
        if count >= testing_size:
            break

    print('Recall@{}: {}'.format(k, recall / testing_size))

Recall@3: 0.0
Recall@5: 0.0
Recall@10: 0.0
Recall@20: 0.0
Recall@50: 0.0
